In [216]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.activations import relu, sigmoid
from keras.optimizers import Adam
from keras.layers import Dense, Input, Dropout
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [217]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

# EDA in Oblig1 Classification notebook

In [218]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)

    df['Embarked'].fillna('S', inplace=True)

    return  df

df = handle_null_median(df)
df['Sex'] = df['Sex'].replace(['female', 'male'], [0,1])
df['FamilyMembersCount'] = df['Parch'] + df['SibSp'] + 1
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

In [219]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    df = df.drop('Age', axis=1)

    # Could also create bins for fare, but not sure Fare is needed.
    return df

df = bins(df)

df['Pclass'] = df['Pclass'].astype('category')
df['Sex'] = df['Sex'].astype('category')
df['Embarked'] = df['Embarked'].astype('category')
df['AgeGroup'] = df['AgeGroup'].astype('category')

df = df.drop(['Fare'], axis=1)

df.head()

,Survived,Pclass,Sex,Embarked,FamilyMembersCount,AgeGroup
0,0,3,1,S,2,1
1,1,1,0,C,2,2
2,1,3,0,S,1,1
3,1,1,0,S,2,2
4,0,3,1,S,1,2


In [220]:
features = df.drop('Survived',axis=1)
targets = df['Survived']

targets_onehot = pd.get_dummies(targets)
features_onehot = pd.get_dummies(features)
targets_onehot = targets_onehot.astype('float32')
features_onehot = features_onehot.astype('float32')

features_onehot.head()

,FamilyMembersCount,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Embarked_C,Embarked_Q,Embarked_S,AgeGroup_0,AgeGroup_1,AgeGroup_2,AgeGroup_3,AgeGroup_4
0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [221]:
# features_onehot = normalize(features_onehot)

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.25, random_state=42)

In [222]:
train_features_tensor = tf.convert_to_tensor(train_features)
test_features_tensor = tf.convert_to_tensor(test_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_targets_tensor = tf.convert_to_tensor(test_targets)

# NN Architecture 1

In [177]:
model1 = Sequential()

model1.add(Input(14))

model1.add(Dense(30, activation='relu'))
model1.add(Dense(30, activation='relu'))
model1.add(Dense(15, activation='relu'))

model1.add(Dense(2, activation='sigmoid'))

In [178]:
opt1 = Adam(learning_rate=0.01)
model1.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [179]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=200, mode='auto')
model1.fit(train_features_tensor, train_targets_tensor, epochs=250, batch_size=10, callbacks=[callback])

Epoch 1/250
67/67 [==============================] - 0s 2ms/step - loss: 0.5363 - accuracy: 0.7515
Epoch 2/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4549 - accuracy: 0.7994
Epoch 3/250
67/67 [==============================] - 0s 2ms/step - loss: 0.4363 - accuracy: 0.7979
Epoch 4/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4342 - accuracy: 0.8084
Epoch 5/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4207 - accuracy: 0.8069
Epoch 6/250
67/67 [==============================] - 0s 2ms/step - loss: 0.4215 - accuracy: 0.8144
Epoch 7/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4243 - accuracy: 0.8009
Epoch 8/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4152 - accuracy: 0.8054
Epoch 9/250
67/67 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.8084
Epoch 10/250
67/67 [==============================] - 0s 1ms/step - loss: 0.4173 - accuracy: 0.8293
Epoch 11/

In [180]:
model1.evaluate(train_features_tensor, train_targets_tensor)

21/21 [==============================] - 0s 647us/step - loss: 0.3234 - accuracy: 0.8533


[0.3233720064163208, 0.8532934188842773]

In [181]:
model1.evaluate(test_features_tensor, test_targets_tensor)

7/7 [==============================] - 0s 848us/step - loss: 0.6104 - accuracy: 0.8251


[0.6104143857955933, 0.8251121044158936]

## Atempt at Automatic Hyperparameter tuning

In [223]:
def create_model1(learning_rate, layer1_neurons, layer2_neurons, layer3_neurons, activation_function):
    model = Sequential([
        Input(15),
        Dense(layer1_neurons, activation=activation_function),
        Dense(layer2_neurons, activation=activation_function),
        Dense(layer3_neurons, activation=activation_function),
        Dense(2, activation='sigmoid')
    ])

    opt1 = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [224]:
classifier = KerasClassifier(create_model1, epochs=200, batch_size=10)
params = {
    'learning_rate':[0.01,0.1,0.2,0.3],
    'layer1_neurons':[15,30,60],
    'layer2_neurons':[15,30,60],
    'layer3_neurons':[15,30,60],
    'activation_function':[relu, sigmoid]
}
grid = GridSearchCV(classifier, params)
grid.fit(train_features_tensor, train_targets_tensor)

C:\Users\marku\AppData\Local\Temp\ipykernel_23464\940212376.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(create_model1, epochs=200, batch_size=10)


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
       199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
       225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
       238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
       251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
       264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
       277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289

This architecture performs slightly worse than the on in the 2nd assignment

# NN Architecture 2

In [211]:
model2 = Sequential()

model2.add(Input(14))

model2.add(Dense(30, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(15, activation='relu'))
model2.add(Dense(2, activation='sigmoid'))

In [212]:
opt1 = Adam()
model2.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

In [213]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=200, mode='auto')
model2.fit(train_features_tensor, train_targets_tensor, epochs=250, batch_size=15, callbacks=[callback])

Epoch 1/250
45/45 [==============================] - 0s 1ms/step - loss: 0.6919 - accuracy: 0.5749
Epoch 2/250
45/45 [==============================] - 0s 2ms/step - loss: 0.5944 - accuracy: 0.6841
Epoch 3/250
45/45 [==============================] - 0s 2ms/step - loss: 0.5321 - accuracy: 0.7814
Epoch 4/250
45/45 [==============================] - 0s 1ms/step - loss: 0.5069 - accuracy: 0.7680
Epoch 5/250
45/45 [==============================] - 0s 1ms/step - loss: 0.4702 - accuracy: 0.8009
Epoch 6/250
45/45 [==============================] - 0s 2ms/step - loss: 0.4571 - accuracy: 0.8084
Epoch 7/250
45/45 [==============================] - 0s 1ms/step - loss: 0.4555 - accuracy: 0.7979
Epoch 8/250
45/45 [==============================] - 0s 2ms/step - loss: 0.4452 - accuracy: 0.7949
Epoch 9/250
45/45 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.8084
Epoch 10/250
45/45 [==============================] - 0s 1ms/step - loss: 0.4334 - accuracy: 0.8054
Epoch 11/

In [214]:
model2.evaluate(train_features_tensor, train_targets_tensor)

21/21 [==============================] - 0s 705us/step - loss: 0.3498 - accuracy: 0.8443


[0.3498385548591614, 0.8443113565444946]

In [215]:
model2.evaluate(test_features_tensor, test_targets_tensor)

7/7 [==============================] - 0s 667us/step - loss: 0.4859 - accuracy: 0.8161


[0.48585543036460876, 0.8161435127258301]

This model has fewer layers than the previous model and a dropout layer, but performs slightly worse than the previous one.

## Atempt at Automatic Hyperparameter tuning

In [92]:
def create_model2(learning_rate, layer1_neurons, layer2_neurons, activation_function):
    model = Sequential([
        Input(15),
        Dense(layer1_neurons, activation=activation_function),
        Dense(layer2_neurons, activation=activation_function),
        Dense(2, activation='sigmoid')
    ])

    opt1 = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

    return model

classifier = KerasClassifier(create_model2, epochs=200, batch_size=10)
params = {
    'learning_rate':[0.01,0.1,0.2,0.3],
    'layer1_neurons':[15,30,60],
    'layer2_neurons':[15,30,60],
    'activation_function':[relu, sigmoid]
}
grid = GridSearchCV(classifier, params)
grid.fit(train_features_tensor, train_targets_tensor)

C:\Users\marku\AppData\Local\Temp\ipykernel_23464\4214831776.py:14: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(create_model2, epochs=200, batch_size=10)


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got array([134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198,
       199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211,
       212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,
       225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
       238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250,
       251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
       264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
       277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289